In [ ]:

import pandas as pd
import os
from langchain_ollama import OllamaEmbeddings
from langchain_ollama.llms import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate


ModuleNotFoundError: No module named 'pandas'

In [ ]:
CSV_STORAGE_PATH = 'document_store/'
CSV_FILE_PATH = os.path.join(CSV_STORAGE_PATH, 'amazon.csv')  
OUTPUT_CSV_FILE_PATH = os.path.join(CSV_STORAGE_PATH, 'processed_amazon.csv')

EMBEDDING_MODEL = OllamaEmbeddings(model="mistral")
LANGUAGE_MODEL = OllamaLLM(model="mistral")
PROMPT_TEMPLATE = """
Analyze the emotion in the following text and return a single word emotion (e.g., happy, frustrated, excited, disappointed, worried, relieved, neutral, etc.). 

### **Example Output:**
Text: "The product quality was amazing and worth the price!" -> Emotion: Happy
Text: "I was really disappointed with the slow delivery." -> Emotion: Disappointed

Only return the emotion, nothing else.
"""


In [ ]:
def load_csv(file_path):
    df = pd.read_csv(file_path, header=None)
    if df.empty:
        print("The CSV file is empty or incorrectly formatted.")
        return None
    return df


In [ ]:
def extract_emotions(texts):
    emotions = []
    conversation_prompt = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    response_chain = conversation_prompt | LANGUAGE_MODEL
    
    for text in texts:
        response = response_chain.invoke({"user_query": text})
        emotions.append(response.strip())
    
    return emotions


: 

In [ ]:
if os.path.exists(CSV_FILE_PATH):
    df = load_csv(CSV_FILE_PATH)
    if df is not None:
        review_column = df.iloc[:, -1].dropna().astype(str)  
        extracted_emotions = extract_emotions(review_column)
        
        df_emotions = pd.DataFrame({"Review": review_column, "Emotion": extracted_emotions})
        df_final = pd.concat([df, df_emotions[["Emotion"]]], axis=1)
        df_final.to_csv(OUTPUT_CSV_FILE_PATH, index=False)
        
        print("✅ CSV file processed successfully! Emotions extracted and saved.")
        print(df_final)
else:
    print(f"❌ File not found: {CSV_FILE_PATH}. Please ensure 'amazon.csv' is in the 'document_store/' folder.")